In [1]:
import sys
import pandas as pd
import numpy as np
import nltk
import re


In [2]:
from modules.load_data import load_data, DocID
from modules.create_word_pos_list import create_word_pos_list
from modules.search import search
from modules.count_pos_patterns import count_pos_patterns
from modules.predict import predict

In [3]:
df = load_data()

In [4]:
df

,author,message
0,Q,"\n\nHowever, there are frequent situations whe..."
1,K1,Download\n\nSource\n\nPDF\nActions\n Copy Pr...
2,K2,\n\nWith the rapid growth of the information c...


## Create a list of tuples of (word, POS)

In [5]:
df.insert(2,"word_pos_list",[ create_word_pos_list(message) for message in df['message']],True)

In [6]:
df['word_pos_list'][2][:3]

[('With', 'IN'), ('the', 'DT'), ('rapid', 'JJ')]

# Search

In [16]:
documents = df['word_pos_list']
search(documents, "In th.* NN", datasets=[DocID.Q, DocID.K2, DocID.K1])

query :  ['In', 'th.*', 'NN']
Q :  task and resource manage- ment . In this work , we consider the 
Q :  again ( for example , driving in the area because of some business 
Q :  , where the user is involved in the activity , and the activity 
Q :  discover major entities to be modeled in the system and their key relationships 
Q :  . The condition of the reminder in this system is mainly determined by 
Q :  . Templates are reminder configurations stored in the system for future use to 
Q :  a regular plan to be included in the calendar or time scheduler might 
Q :  to synchronize the geofence coordinates created in the database with the cloud . 
K2:  , thus lowering the energy consumed in the cooling systems . However , 
K2:  , thus lowering the energy consumption in the cooling systems . 4 % 
K2:  As one of the key components in the transceiver , electro-optic ( EO 
K2:  energy-consuming temperature stabilization unit is deployed in the system . The simultaneous fulfilments 
K2:  , th

In [8]:
pos_patterns, pos_freq_vectors = count_pos_patterns(documents, window_size=3)
df['pos_freq'] = pos_freq_vectors

In [9]:
# show the top pattern of Q
doc_id = DocID.Q
max_idx = pos_freq_vectors[doc_id].index(max(pos_freq_vectors[doc_id]))
pos_patterns[max_idx]

'the JJ NN'

In [10]:
len(df)

3

In [11]:
from modules.predict import predict
Q_df = df.loc[DocID.Q]
K_df = df.loc[DocID.K1:DocID.K2].reset_index(drop=True)

In [12]:
K_df

,author,message,word_pos_list,pos_freq
0,K1,Download\n\nSource\n\nPDF\nActions\n Copy Pr...,"[(Download, NNP), (Source, NNP), (PDF, NNP), (...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,K2,\n\nWith the rapid growth of the information c...,"[(With, IN), (the, DT), (rapid, JJ), (growth, ...","[0, 0, 0, 0, 0, 0, 5, 7, 0, 0, 1, 0, 7, 0, 0, ..."


In [13]:
predict(Q_df, K_df, target='pos_freq', _list=pos_patterns)

==================== Top 20 ====================
Suspected : {'K1', 'K2'}
Q  :the JJ NN | the NN IN | . CD . | a NN IN | a JJ NN | of JJ NN | to VB DT | be VBN IN | by DT NN | of JJ NNS | . NN CD | the NNS IN | to VB VBN | [ CD NN | . DT NN | Figure CD . | the NN NN | the NN VBZ | be JJ IN | [ CD NNP | 


K1 : [ CD NN | the NN IN | a JJ NN | in DT NN | of NNP NNP | s NNP NNP | the JJ NN | music NN NN | be VBN IN | ’ NN NNP | can VB VBN | of NNS IN | the NN CC | to VB DT | of JJ NN | and JJ NN | , IN DT | Children NNP NN | . IN DT | [ CD NNP | 
Score : 8
Shared list Q-K1 : be VBN IN | of JJ NN | [ CD NN | the JJ NN | to VB DT | a JJ NN | the NN IN | [ CD NNP | 

K2 : the JJ NN | the NN NN | ( NNP ) | a JJ NN | of DT NN | the NN IN | . DT JJ | the JJ JJ | the NNP NN | of DT JJ | is VBN IN | up TO CD | , DT NN | , DT JJ | Fig . CD | . IN DT | are VBN IN | in DT NN | to VB DT | in NNP . | 
Score : 5
Shared list Q-K2 : the JJ NN | to VB DT | a JJ NN | the NN NN | the NN IN | 
--------------

'K1'

In [14]:
nparray = np.array(df['pos_freq'][DocID.K1])
num = np.count_nonzero(nparray)


In [15]:
arr = np.array([40, 10, 30, 20])

# argsortを使用してインデックスを取得
sorted_indices = np.argsort(arr)

print("ソート後のインデックス:", sorted_indices)
print("ソート後の配列:", arr[sorted_indices])

ソート後のインデックス: [1 3 2 0]
ソート後の配列: [10 20 30 40]
